In [1]:
import os
import random

import numpy as np
import quaternion

import xml.etree.cElementTree as ET
import xml.dom.minidom as minidom

from deap import base
from deap import creator
from deap import gp
from deap import tools
from deap import algorithms

from dm_control import mujoco
from dm_control.suite import base as control_base
from dm_control.suite import common
from dm_control.utils import containers
from dm_control.utils import rewards
from dm_control.rl import control
from dm_control.rl.control import PhysicsError

import matplotlib.pyplot as plt
from IPython.display import clear_output
import collections
from skimage import io, transform

from deap import base, creator, gp, tools
import operator
from genome_synth import Genesis, prettify, Physics, Avoid

In [2]:
gen = Genesis()
print(gen.mk_leaf())
pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(gen.combine_trees, 2)
#pset.addEphemeralConstant("mk_leaf", gen.mk_leaf)
pset.addTerminal(gen.mk_leaf)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr_init", gp.genFull, pset=pset, min_=2, max_=4)

toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def get_model_and_assets():
    curpath = os.getcwd()
    return common.read_model(curpath + '/dust.xml'), common.ASSETS

def capture(_savename, individuals):
    imnames = set()
    picidx = 0
    for individual in individuals:
        creation = gp.compile(individual, pset)
        tree = ET.fromstring(prettify(gen.mk_model(creation)))
        tree = ET.ElementTree(tree)
        tree.write('dust.xml')
        _DEFAULT_TIME_LIMIT = 10
        _CONTROL_TIMESTEP = .04
        display_stride = 1 / .04 // 24

        genesis_physics = Physics.from_xml_string(*get_model_and_assets())
        genesis_physics.set_gen(gen)
        genesis_task = Avoid()
        genesis_env = control.Environment(genesis_physics, 
                                         genesis_task,
                                         control_timestep=_CONTROL_TIMESTEP,
                                         time_limit=_DEFAULT_TIME_LIMIT)
        action_spec = genesis_env.action_spec()
        observation_spec = genesis_env.observation_spec()

        time_step = genesis_env.reset()
        curtime = 0.0

        

        while(not time_step.last()):
            try:
                action = np.ones(action_spec.shape[0]) * np.sin(curtime)
                time_step = genesis_env.step(action)
                savename = "basedgod_{0:04}.jpg".format(picidx)
                picidx += 1
                imnames.add(savename)
                curtime += _CONTROL_TIMESTEP
                io.imsave(savename, genesis_env.physics.render(480, 640, camera_id='tracking_top'))
            except PhysicsError:
                print('except')
                break
    if os.path.isfile('basedgod.mp4'):
        os.remove('basedgod.mp4')
    !!ffmpeg -f image2 -pattern_type sequence -i "basedgod_%4d.jpg" -qscale:v 0 basedgod.mp4
    for name in imnames:
        os.remove(name)
    
def evalGen(individual):
    creation = gp.compile(individual, pset)
    #print(creation)
    tree = ET.fromstring(prettify(gen.mk_model(creation)))
    tree = ET.ElementTree(tree)
    tree.write('dust.xml')
    _DEFAULT_TIME_LIMIT = 10
    _CONTROL_TIMESTEP = .04
    display_stride = 1 / .04 // 24
    picidx = 0
    imnames = set()
    
    genesis_physics = Physics.from_xml_string(*get_model_and_assets())
    genesis_physics.set_gen(gen)
    genesis_task = Avoid()
    genesis_env = control.Environment(genesis_physics, 
                                     genesis_task,
                                     control_timestep=_CONTROL_TIMESTEP,
                                     time_limit=_DEFAULT_TIME_LIMIT)
    action_spec = genesis_env.action_spec()
    observation_spec = genesis_env.observation_spec()
    
    #print(vars(genesis_env.physics.data))
    
    idx = 0
    
    maxreward = 0.0
    max_energy_penalty = 0.0
    
    time_step = genesis_env.reset()
    curtime = 0.0
    
    while(not time_step.last()):
        try:
            action = np.ones(action_spec.shape[0]) * np.sin(curtime)
            time_step = genesis_env.step(action)
            reward = time_step.reward
            if(maxreward < reward):
                maxreward = reward
            curtime += _CONTROL_TIMESTEP
            idx += 1
        except PhysicsError:
            print('except')
            maxreward -= 500
            break
    return (maxreward,)

toolbox.register("evaluate", evalGen)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=2, max_=4)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)




{'name': '0', 'offset': array([0., 0., 0.]), 'rotation': array([148.92581029, 127.34710622,  91.89551391]), 'dimensions': [0.07523565630688736, 0.015492401013331289, 0.015492401013331289], 'joint_range': 137.21731052223083, 'children': [], 'priority': 0.4633342832939057}


In [4]:
def main():
    pop = toolbox.population(n=128)
    hof = tools.HallOfFame(5)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 15, stats, halloffame=hof)
    return pop, hof, stats

if __name__ == "__main__":
    pop, hoff, _ = main()
    capture("basedgod", hoff)
    
    

gen	nevals	avg      	std     	min	max     
0  	128   	0.0389424	0.158364	0  	0.783759
1  	75    	0.0948576	0.236787	0  	0.790384
2  	69    	0.228466 	0.328936	0  	0.791393
3  	75    	0.390022 	0.374333	0  	0.791404
4  	72    	0.519012 	0.368548	0  	0.791688
5  	55    	0.663403 	0.287475	0  	0.791688
6  	72    	0.650093 	0.302158	0  	0.791688
7  	68    	0.717306 	0.23071 	0  	0.791688
8  	63    	0.704972 	0.247049	0  	0.791701
9  	68    	0.680221 	0.275163	0  	0.791701
10 	72    	0.723473 	0.221833	0  	0.79171 
11 	67    	0.729705 	0.212426	0  	0.794183
12 	65    	0.735891 	0.202377	0  	0.794183
13 	62    	0.748372 	0.180001	0  	0.794183
14 	75    	0.723653 	0.221892	0  	0.794183
15 	70    	0.730307 	0.212604	0  	0.794183
